In [2]:
!pip uninstall numpy
!pip install numpy==1.26.4

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.0.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-99b71e71.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 68.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which i

In [1]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2469526 sha256=282316ac545d8e55da99a1380a9aca6b02fff41d51a06270ace59035b4dbf049
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [27]:
import pandas as pd
from collections import defaultdict
from surprise import Reader, Dataset, SVD, dump, NMF, accuracy, SlopeOne
from surprise.model_selection import cross_validate, train_test_split, KFold, GridSearchCV
# from sklearn.model_selection import train_test_split

In [ ]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


In [4]:
# review_df = pd.read_csv('/content/drive/MyDrive/Kinopoisk/review_df.csv')
# movie_df = pd.read_csv('/content/drive/MyDrive/Kinopoisk/movie_df.csv')
ratings= pd.read_csv('/content/drive/MyDrive/Kinopoisk/datasets/raw/rating_df.csv', usecols=['userId', 'movieId', 'rating'])
# ratings= pd.read_csv('/content/drive/MyDrive/Kinopoisk/train.csv', usecols=['userId', 'movieId', 'rating'])

In [5]:
# ratings = review_df[['authorId', 'movieId', 'type']]
# ratings.loc[:,'type'] = ratings['type'].map({'Позитивный':5, 'Нейтральный':3, 'Негативный':1})
# ratings = ratings[['authorId', 'movieId', 'type']]
# ratings.rename(columns={'type':'rating'}, inplace=True)
ratings

,userId,movieId,rating
0,1.0,342.0,5.0
1,4.0,342.0,4.0
2,5.0,342.0,4.0
3,7.0,342.0,4.0
4,8.0,342.0,5.0
...,...,...,...
10291614,168183.0,28279.0,3.0
10291615,818517.0,28452.0,1.0
10291616,1696664.0,28452.0,3.0
10291617,172902.0,28452.0,3.0


In [6]:
reader = Reader(rating_scale=(1,5))

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings, train_size=0.1, stratify=None)
train.to_csv('/content/drive/MyDrive/Kinopoisk/rating_train_df.csv')
del test
data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)
param_grid = {"n_epochs": [5, 10, 20], "lr_all": [0.002, 0.005], "reg_all": [0.002, 0.005], "n_factors":[50, 100, 200]}

gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae", "fcp"], cv=10, n_jobs=5, joblib_verbose=5)
gs.fit(data)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:   58.5s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:  7.5min
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed: 20.0min
[Parallel(n_jobs=5)]: Done 278 tasks      | elapsed: 45.5min
[Parallel(n_jobs=5)]: Done 360 out of 360 | elapsed: 69.0min finished


In [ ]:
# # best FCP score
print(f'Best FCP score is {gs.best_score["fcp"]}')
# print(gs.best_score["fcp"])

print('Combination of parameters that gave the best FCP score:')
print(gs.best_params["fcp"])

NameError: name 'gs' is not defined

In [ ]:
# # best RMSE score
# print(gs.best_score["mae"])

# # combination of parameters that gave the best RMSE score
# print(gs.best_params["mae"])

0.7234985701408936
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.005, 'n_factors': 50}


In [ ]:
# # best RMSE score
# print(gs.best_score["rmse"])

# # combination of parameters that gave the best RMSE score
# print(gs.best_params["rmse"])

0.9372344873893317
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.005, 'n_factors': 50}


In [7]:
reader = Reader(rating_scale=(1,5))

In [8]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
del ratings
trainset, testset = train_test_split(data, test_size=0.2)

In [9]:
svd=SVD(n_epochs=10, lr_all=0.005, reg_all=0.005, n_factors=50)
svd.fit(trainset)
predictions = svd.test(testset)

In [10]:
print(f"SVD model's RMSE {accuracy.rmse(predictions)}")
print(f"SVD model's MAE {accuracy.mae(predictions)}")
print(f"SVD model's FCP {accuracy.fcp(predictions)}")

RMSE: 0.8517
SVD model's RMSE 0.851695742626227
MAE:  0.6487
SVD model's MAE 0.6486865373084619
FCP:  0.6863
SVD model's FCP 0.6862583585798523


In [ ]:
user_est_true = defaultdict(list)
for uid, _, true_r, est, _ in predictions:
  user_est_true[uid].append((est, true_r))

In [11]:
predictions[1]

Prediction(uid=82865.0, iid=3786.0, r_ui=2.0, est=2.4331501512493245, details={'was_impossible': False})

In [12]:
svd_prediction = pd.DataFrame(predictions)

In [33]:
svd_prediction.drop(columns=['AE', 'SE'])

,uid,iid,r_ui,est,details
0,57963.0,3915.0,4.0,3.330335,{'was_impossible': False}
1,82865.0,3786.0,2.0,2.433150,{'was_impossible': False}
2,13041.0,670.0,0.5,2.483916,{'was_impossible': False}
3,142641.0,2022.0,3.0,3.235675,{'was_impossible': False}
4,99939.0,438.0,4.0,4.107584,{'was_impossible': False}
...,...,...,...,...,...
2058319,78831.0,6695.0,5.0,3.780742,{'was_impossible': False}
2058320,7020.0,9249.0,4.0,3.702302,{'was_impossible': False}
2058321,74658.0,7908.0,4.0,3.562842,{'was_impossible': False}
2058322,77925.0,15263.0,4.0,3.876784,{'was_impossible': False}


In [22]:
svd_prediction['AE'] = abs(svd_prediction['r_ui']-svd_prediction['est'])
svd_prediction['SE'] = svd_prediction['AE']*svd_prediction['AE']

In [26]:
hybrid_mae = svd_prediction['AE'].sum()/svd_prediction.shape[0]
hybrid_rmse = (svd_prediction['SE'].sum()/svd_prediction.shape[0])**0.5

In [28]:
slopeone=SlopeOne()
slopeone.fit(trainset)
# del trainset

predictions = slopeone.test(testset)

In [29]:
slopeone_prediction = pd.DataFrame(predictions)

In [36]:
merged_prediction = pd.merge(svd_prediction,slopeone_prediction, on = ['uid', 'iid'])

In [38]:
merged_prediction['est'] = (merged_prediction['est_x'] + merged_prediction['est_y']) / 2

In [40]:
merged_prediction['AE'] = abs(merged_prediction['r_ui_x']-merged_prediction['est'])
merged_prediction['SE'] = merged_prediction['AE']*merged_prediction['AE']
merged_prediction

,uid,iid,r_ui_x,est_x,details_x,AE,SE,r_ui_y,est_y,details_y,est
0,57963.0,3915.0,4.0,3.330335,{'was_impossible': False},0.552006,0.304711,4.0,3.565652,{'was_impossible': False},3.447994
1,82865.0,3786.0,2.0,2.433150,{'was_impossible': False},0.320723,0.102863,2.0,2.208295,{'was_impossible': False},2.320723
2,13041.0,670.0,0.5,2.483916,{'was_impossible': False},2.256621,5.092339,0.5,3.029327,{'was_impossible': False},2.756621
3,142641.0,2022.0,3.0,3.235675,{'was_impossible': False},0.269820,0.072803,3.0,3.303966,{'was_impossible': False},3.269820
4,99939.0,438.0,4.0,4.107584,{'was_impossible': False},0.227736,0.051864,4.0,4.347889,{'was_impossible': False},4.227736
...,...,...,...,...,...,...,...,...,...,...,...
2058323,78831.0,6695.0,5.0,3.780742,{'was_impossible': False},1.440117,2.073938,5.0,3.339024,{'was_impossible': False},3.559883
2058324,7020.0,9249.0,4.0,3.702302,{'was_impossible': False},0.267418,0.071512,4.0,3.762862,{'was_impossible': False},3.732582
2058325,74658.0,7908.0,4.0,3.562842,{'was_impossible': False},0.454411,0.206490,4.0,3.528335,{'was_impossible': False},3.545589
2058326,77925.0,15263.0,4.0,3.876784,{'was_impossible': False},0.201087,0.040436,4.0,3.721042,{'was_impossible': False},3.798913


In [64]:
hybrid_mae = merged_prediction['AE'].sum()/merged_prediction.shape[0]
hybrid_rmse = (merged_prediction['SE'].sum()/merged_prediction.shape[0])**0.5

In [65]:
print(f"Hybrid model's RMSE {hybrid_rmse}")
print(f"Hybrid model's MAE {hybrid_mae}")


Hybrid model's RMSE 0.8479321437427009
Hybrid model's MAE 0.646395739849232
